In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_x1 = pd.read_csv('../data/train_ssvm_xy.csv')
train_x1.shape

(10000, 160)

In [3]:
train_x2 = pd.read_csv('../data/train_xy.csv')
train_x2.shape

(15000, 160)

In [4]:
test = pd.read_csv('../data/test_all.csv')
test.shape

(10000, 159)

In [5]:
train_x11 = train_x1.drop(['cust_id','cust_group','y'],axis=1)
train_x22 = train_x2.drop(['\ufeffcust_id','cust_group','y'],axis=1)
train_x = pd.concat([train_x11, train_x22])
train_x.shape

(25000, 157)

In [6]:
test_x = test.drop(['cust_id','cust_group'],axis=1)
test_x.shape

(10000, 157)

In [7]:
x = pd.concat([train_x, test_x])
x.shape

(35000, 157)

In [8]:
train_y1 = train_x1['y']
train_y2 = train_x2['y']
Y_train = train_y1.append(train_y2)
Y_train.shape

(25000,)

In [9]:
for i in range(96,158):
    col = 'x'+'_'+str(i)
    if col in x.columns.values:
        dummies_df = pd.get_dummies(x[col]).rename(columns=lambda x: col +'_'+ str(x))
        x = pd.concat([x, dummies_df], axis=1)
print(x.shape)

(35000, 364)


In [10]:
train_X = x[0:25000]
test_X = x[25000:35000]
print(train_X.shape)
print(test_X.shape)

(25000, 364)
(10000, 364)


In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import metrics  #accuracy_score,recall_score,f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import cross_val_score
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.svm import LinearSVC
from sklearn import linear_model
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

from keras.models import Model
from keras.layers import Dense, Input

C:\Program Files\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [12]:
# encoding_dim = 600
# input_dim = Input(shape=(364,))

# encoded = Dense(364, activation='linear')(input_dim)
# # encoded = Dense(300, activation='relu')(encoded)
# # encoded = Dense(32, activation='relu')(encoded)
# encoder_output = Dense(encoding_dim)(encoded)

# decoded = Dense(600, activation='relu')(encoder_output)
# # decoded = Dense(64, activation='relu')(decoded)
# # decoded = Dense(128, activation='relu')(decoded)
# decoded = Dense(364, activation='tanh')(decoded)

# autoencoder = Model(inputs=input_dim, outputs=decoded)

# encoder = Model(inputs=input_dim, outputs=encoder_output)

# autoencoder.compile(optimizer='adam', loss='mse')
# # training
# autoencoder.fit(train_X.values, train_X.values, epochs=20, batch_size=150, shuffle=True)

In [13]:
# new_train_feature = encoder.predict(train_X.values)
# new_test_feature = encoder.predict(test_X.values)
# print(new_train_feature.shape)
# print(new_test_feature.shape)

In [14]:
X_train,X_val,y_train,y_val= train_test_split(train_X,Y_train,test_size=0.2,random_state=2)

In [15]:
tuned_parameters= [{'n_estimators':range(100,300,500),
                  'max_depth':range(4,8,12),
                  'learning_rate':[0.01, 0.1]
                  }]
clf = GridSearchCV(GradientBoostingClassifier(), tuned_parameters, cv=5, scoring='roc_auc')
clf.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'max_depth': range(4, 8, 12), 'n_estimators': range(100, 300, 500), 'learning_rate': [0.01, 0.1]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [18]:
predictions = clf.predict_proba(X_val)
pre = predictions[:,1]
val_auc = metrics.roc_auc_score(y_val,pre)#验证集上的auc值
print(val_auc)  

0.805375686875


In [19]:
preds = clf.predict_proba(test_X)
pred = preds[:,1]
pred.shape

(10000,)

In [20]:
Submission = pd.DataFrame({'cust_id': test['cust_id'], 'pred_prob': pred})
Submission.to_csv('../result/semi_gbdt.csv',index=False)